# Creating model

In [9]:
from keras import layers
from keras.models import Sequential
import keras_tuner as kt
from keras import optimizers

In [10]:
def build_model(hp):
    model = Sequential()

    model.add(layers.Conv2D(32, (3,3), activation="relu", input_shape = (150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(64, (3,3), activation="relu" ))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), activation="relu" ))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Conv2D(128, (3,3), activation="relu" ))
    model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())

    model.add(layers.Dense(units=hp.Choice("layer size",[128,256,512]), activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))

    model.compile(
        loss="binary_crossentropy", 
        optimizer=optimizers.RMSprop(learning_rate= 1e-4), metrics="accuracy"
    )
    
    return model

    



In [11]:
keras_tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    overwrite=True,
    max_trials=3 ,
    directory= "./models/parameter_tuning",
    project_name = "parameter_tuning"
)

In [12]:
train_directory = "../datasets/catdog/train/"
validation_directory = "../datasets/catdog/validation/"

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(train_directory, target_size=(150,150), batch_size=20, class_mode="binary")

validation_genarator = validation_datagen.flow_from_directory(validation_directory, target_size=(150,150),batch_size=20, class_mode="binary")

Found 1999 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [15]:
keras_tuner.search_space_summary()
keras_tuner.search(
    train_generator,
    validation_data = validation_genarator, 
    epochs=50
    )

best_model = keras_tuner.get_best_models()[0]



Trial 3 Complete [00h 11m 44s]
val_accuracy: 0.7590000033378601

Best val_accuracy So Far: 0.7590000033378601
Total elapsed time: 00h 34m 03s
INFO:tensorflow:Oracle triggered exit


In [16]:
best_model.save("best_model.h5")